In [ ]:
# install libraries on aws linux:

# ! pip install healpy

# ! pip install camb

In [ ]:
# test installs
# import camb
# import healpy

In [ ]:
# https://github.com/ixkael/Prob-tools/blob/master/notebooks/The%20CMB%20as%20a%20Gaussian%20Process.ipynb

In [ ]:
%matplotlib inline
%config IPython.matplotlib.backend = 'retina'
%config InlineBackend.figure_format = 'retina'

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc
from cycler import cycler

import numpy as np

rc("font", family="serif", size=14)
rc("text", usetex=False)
matplotlib.rcParams['lines.linewidth'] = 2
matplotlib.rcParams['patch.linewidth'] = 2
matplotlib.rcParams['axes.prop_cycle'] =\
    cycler("color", ['k', 'c', 'm', 'y'])
matplotlib.rcParams['axes.labelsize'] = 16

import healpy as hp

import camb
from camb import model, initialpower

In [ ]:
nside = 512  # Healpix parameter, giving 12*nside**2 equal-area pixels on the sphere.
lmax = 3*nside # band-limit. Should be 2*nside < lmax < 4*nside to get information content.

Reference: https://camb.readthedocs.io/en/latest/model.html

* set_cosmology(H0: float | None = None, ombh2=0.022, omch2=0.12, omk=0.0, cosmomc_theta: float | None = None, thetastar: float | None = None, neutrino_hierarchy: str | int = 'degenerate', num_massive_neutrinos=1, mnu=0.06, nnu=3.044, YHe: float | None = None, meffsterile=0.0, standard_neutrino_neff=3.044, TCMB=2.7255, tau: float | None = None, zrei: float | None = None, deltazrei: float | None = None, Alens=1.0, bbn_predictor: None | str | BBNPredictor = None, theta_H0_range=(10, 100))   

-------------      

* H0 – (float64) Hubble parameter is km/s/Mpc units  
* H0 – Hubble parameter today in km/s/Mpc. Can leave unset and instead set thetastar or cosmomc_theta (which solves for the required H0).

* ombh2 – physical density in baryons
* omch2 – physical density in cold dark matter

* mnu – sum of neutrino masses (in eV). Omega_nu is calculated approximately from this assuming neutrinos non-relativistic today; i.e. here is defined as a direct proxy for Omega_nu. Internally the actual physical mass is calculated from the Omega_nu accounting for small mass-dependent velocity corrections but neglecting spectral distortions to the neutrino distribution. Set the neutrino field values directly if you need finer control or more complex neutrino models.

* omk – Omega_K curvature parameter

* tau – optical depth; if None and zrei is None, current Reion settings are not changed

(many more parameters at https://camb.readthedocs.io/en/latest/model.html)

Reference: https://en.wikipedia.org/wiki/Lambda-CDM_model

Dark matter constitutes about 26.5%[8] of the mass–energy density of the universe. The remaining 4.9%[8] comprises all ordinary matter observed as atoms, chemical elements, gas and plasma, the stuff of which visible planets, stars and galaxies are made. The great majority of ordinary matter in the universe is unseen, since visible stars and gas inside galaxies and clusters account for less than 10% of the ordinary matter contribution to the mass–energy density of the universe.[9]

Also, the energy density includes a very small fraction (~ 0.01%) in cosmic microwave background radiation, and not more than 0.5% in relic neutrinos. Although very small today, these were much more important in the distant past, dominating the matter at redshift > 3200.

In [ ]:
ombh2=0.022 # baryon density
omch2=0.122 # cold dark matter density
mnu=0.06 # mnu – sum of neutrino masses (in eV)

print(4.9/26.5)

print(ombh2 / omch2)
# print((ombh2+mnu) / omch2)

In [ ]:
#def cosmology_simulation(omch2_in=0.122):
def cosmology_simulation(omch2_in):
    '''
    Parameters:
    -----------
    omch2: Float. Physical density in cold dark matter. Default omch2=0.122
    
    Returns:
    --------
    ells: Numpy array
    Ells: Numpy array
    Dells: Numpy Array
    
    References:
    -----------
    https://camb.readthedocs.io/en/latest/model.html
    http://camb.readthedocs.io/en/latest/CAMBdemo.html
    https://github.com/ixkael/Prob-tools/blob/master/notebooks/The%20CMB%20as%20a%20Gaussian%20Process.ipynb
    '''
    pars = camb.CAMBparams()
    #pars.set_cosmology(H0=67.5, ombh2=0.022, omch2=0.122, mnu=0.06, omk=0, tau=0.06)
    pars.set_cosmology(H0=67.5, ombh2=0.022, omch2=omch2_in, mnu=0.06, omk=0, tau=0.06)
    pars.InitPower.set_params(ns=0.965, r=0)
    pars.set_for_lmax(lmax, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    powers = results.get_cmb_power_spectra(pars)
    totCL = powers['total']
    unlensedCL = powers['unlensed_scalar']

    ells = np.arange(totCL.shape[0])
    Dells = totCL[:, 0]
    Cells = Dells * 2*np.pi / ells / (ells + 1)  # change of convention to get C_ell
    Cells[0:2] = 0
    
    return ells, Dells, Cells, omch2_in

In [ ]:
def visualize_cmb_power_spectrum(ells, Dells, Cells):
    '''
    Reference:
    https://github.com/ixkael/Prob-tools/blob/master/notebooks/The%20CMB%20as%20a%20Gaussian%20Process.ipynb
    '''
    fig, axs = plt.subplots(1, 2, figsize=(10, 4))
    axs[0].plot(ells, Dells)
    axs[1].plot(ells, Cells)
    axs[0].set_xlabel(r'$\ell$')
    axs[1].set_xlabel(r'$\ell$')
    axs[0].set_ylabel(r'$\frac{\ell (\ell+1)}{2\pi} C_\ell$ [$\mu$K^2]')
    axs[1].set_ylabel(r'$C_\ell$ [$\mu$K^2]')
    axs[1].set_yscale('log')
    axs[0].set_xlim([2, lmax])
    axs[1].set_xlim([2, lmax])
    fig.tight_layout()

In [ ]:
# ells, Dells, Cells, omch2_in = cosmology_simulation()
# visualize_cmb_power_spectrum(ells, Dells, Cells)

In [ ]:
# print(Dells.shape, Cells.shape)

In [ ]:
#type(Cells)
#type(Dells)
#type(ells)

In [ ]:
# omch2_in = 0.122
# omch2_in = 0.0

#str(omch2_in)[2:len(str(omch2_in))]

##str(omch2_in)[2:0]

In [ ]:
def simulate_cmb_temperature_anisotropy(Cells, nside, lmax):
    '''
    '''
    cmbmap = hp.synfast(Cells, nside, 
                        lmax=lmax, mmax=None, alm=False, pol=False, 
                        pixwin=False, fwhm=0.0, sigma=None, new=False, verbose=True)
    
    return cmbmap

In [ ]:
def create_cmb_temperature_anisotropy_skymap(cmbmap, classlabel_img, cnt_img):
#def create_cmb_temperature_anisotropy_skymap(Cells, classlabel_img, cnt_img):
    '''
    Parameters:
    -----------
    Cells: Numpy array generated by Camb
    '''
    #cmbmap = hp.synfast(Cells, nside, 
    #                    lmax=lmax, mmax=None, alm=False, pol=False, 
    #                    pixwin=False, fwhm=0.0, sigma=None, new=False, verbose=True)
    
    # name_img = classlabel_img + '_cmb_omch2_' + str(omch2_in) + '_2023-07-30_' + str(cnt_img)+ '.jpg'
    # name_img = classlabel_img + '_cmb_omch2_' + str(omch2_in)[2:len(str(omch2_in))] + '_2023-07-30_' + str(cnt_img)+ '.jpg'
    name_img = classlabel_img + '_' + str(cnt_img) + '_cmb_omch2_' + str(omch2_in)[2:len(str(omch2_in))] + '_20230730' + '.jpg'
    print(name_img)
    
    plt.figure()
    hp.mollview(cmbmap)
    # plt.savefig("output3.jpg")
    # plt.savefig('./data_out_1/' + name_img)
    # plt.savefig('./data_out_2/' + name_img)
    
    # plt.savefig('./data_out_class1a/' + name_img)
    plt.savefig('./data_out_class0a/' + name_img)

In [ ]:
# omch2_in=0.122 # class 1
omch2_in=0.0 # class 0

# ells, Dells, Cells, omch2_in = cosmology_simulation(omch2_in=0.122)
ells, Dells, Cells, omch2_in = cosmology_simulation(omch2_in=omch2_in)

# cmbmap = simulate_cmb_temperature_anisotropy(Cells, nside, lmax)

# create_cmb_temperature_anisotropy_skymap(Cells, classlabel_img, cnt_img)
# create_cmb_temperature_anisotropy_skymap(Cells, '1', 8)
# create_cmb_temperature_anisotropy_skymap(cmbmap, classlabel_img, cnt_img)
# create_cmb_temperature_anisotropy_skymap(cmbmap, '1', 23)

# for i in range(0,10):
for i in range(0,100):
    print(i)
    cmbmap = simulate_cmb_temperature_anisotropy(Cells, nside, lmax)
    # create_cmb_temperature_anisotropy_skymap(cmbmap, '1', i)
    create_cmb_temperature_anisotropy_skymap(cmbmap, '0', i)

In [ ]:
print(omch2_in)
visualize_cmb_power_spectrum(ells, Dells, Cells)